In [1]:

#Shaun Jacob Varghese
#Varun Ram S


import pandas as pd
import numpy as np
import cv2 as cv2
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention
from keras.layers.merge import concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model
!nvidia-smi

Wed Aug 31 22:17:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8     2W /  N/A |    273MiB /  4096MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from keras.datasets import cifar10
(trainx,trainy),(testx,testy) = cifar10.load_data()

print("Train of X and Y is:",(trainx.shape),(trainy.shape))

Train of X and Y is: (50000, 32, 32, 3) (50000, 1)


In [3]:

from keras.datasets import cifar10
(trainx,trainy),(testx,testy) = cifar10.load_data()

print("Train of X and Y is:",(trainx.shape),(trainy.shape))

Train of X and Y is: (50000, 32, 32, 3) (50000, 1)


In [4]:
trainx = trainx.astype('float32') 
testx = testx.astype('float32') 
trainx = trainx / 255.0 
testx = testx / 255.0

from keras.utils import np_utils 
trainy = np_utils.to_categorical(trainy) 
testy = np_utils.to_categorical(testy) 
num_classes = testy.shape[1]




In [5]:
temp=Input(shape=(64,64,1))

c1=keras.layers.Conv2D(32, (7, 7), input_shape=(32,32,7), activation='relu', padding='same')(temp)
p1=keras.layers.MaxPool2D(pool_size=(2, 2))(c1)


x1=keras.layers.Conv2D(64,(1,1),input_shape=(64,64,1), activation='relu', padding='same')(p1)
x2=keras.layers.Conv2D(64,(3,3),input_shape=(64,64,3), activation='relu', padding='same')(p1)
x3=keras.layers.Conv2D(64,(5,5),input_shape=(64,64,5), activation='relu', padding='same')(p1)
x4=keras.layers.MaxPool2D(pool_size=(2,1))(p1)
x5=keras.layers.Conv2D(64,(1,1))(x4)

In [6]:
from scipy.special import softmax

In [7]:
#We will flatten after getting I1 to I4
y1=keras.layers.Flatten()(x1)
y2=keras.layers.Flatten()(x2)
y3=keras.layers.Flatten()(x3)
y5=keras.layers.Flatten()(x5)
#P(H,W,C) ppart of the research paper
par1_1=keras.layers.Dense(64,activation='relu')(y1)
par1_2=keras.layers.Dense(64,activation='relu')(y1)
par1_3=keras.layers.Dense(64,activation='relu')(y1)

par2_1=keras.layers.Dense(64,activation='relu')(y2)
par2_2=keras.layers.Dense(64,activation='relu')(y2)
par2_3=keras.layers.Dense(64,activation='relu')(y2)

par3_1=keras.layers.Dense(64,activation='relu')(y3)
par3_2=keras.layers.Dense(64,activation='relu')(y3)
par3_3=keras.layers.Dense(64,activation='relu')(y3)

par4_1=keras.layers.Dense(64,activation='relu')(y5)
par4_2=keras.layers.Dense(64,activation='relu')(y5)
par4_3=keras.layers.Dense(64,activation='relu')(y5)


In [16]:
P1at = tf.matmul(par1_2,par1_3)
sm1_1 = tf.keras.activations.softmax(P1at, axis=-1)
sm1_2 = tf.matmul(par1_1,sm1_1)
tp= tf.keras.layers.Add()([sm1_2,x1])

P2at = tf.matmul(par2_2,par2_3)
sm2_1 = tf.keras.activations.softmax(P2at, axis=-1)
sm2_2 = tf.matmul(par2_1,sm2_1)
tp1= tf.keras.layers.Add()([sm2_2,x2])

P3at = tf.matmul(par3_2,par3_3)
sm3_1 = tf.keras.activations.softmax(P3at, axis=-1)
sm3_2 = tf.matmul(par3_1,sm3_1)
tp2= tf.keras.layers.Add()([sm3_2,x3])

P4at =tf.matmul(par4_2,par4_3)
sm4_1 = tf.keras.activations.softmax(P4at, axis=-1)
sm4_2 = tf.matmul(par4_1,sm4_1)
tp3= tf.keras.layers.Add()([sm4_2,x5])

fin1=keras.layers.Flatten()(tp)
fin2=keras.layers.Flatten()(tp1)
fin3=keras.layers.Flatten()(tp2)
fin4=keras.layers.Flatten()(tp3)


In [19]:
output = concatenate([fin1,fin2,fin3,fin4])
MSCA = Model(inputs=temp, outputs=output)

In [20]:
print(MSCA.summary())


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   1600        ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 32, 32, 32)   0           ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d_1 (MaxPooling2D)  (None, 16, 32, 32)  0           ['max_pooling2d[0][0]']          
                                                                                            

INFO:tensorflow:Assets written to: C:\Users\shaun\Documents\Deep learning\assets
